In [1]:
#!pip install --upgrade datasets


TODO:
*   Add functionality to treat big files. Split them into chunks-
*   Start trials with live speech functionality trasncription.

In [2]:
import gradio as gr
from transformers import pipeline
import numpy as np
from datasets import load_dataset


models = ["openai/whisper-tiny.en","openai/whisper-small.en", "openai/whisper-medium.en"]
ds = load_dataset("hf-internal-testing/librispeech_asr_dummy", "clean", split="validation")

#load samples
def prepare_audio_samples(ds, num_samples=5):
  """
  Look for ``num_samples`` in the ``ds`` given.
  """
  return [[ds[i]['file']] for i in range(num_samples)]  # Adjust index range as needed

def pipe(model, chunk_lenght=30):
  """
  Note about chunking from HF: The Whisper model is intrinsically designed to work on audio samples of up to 
  30s in duration. However, by using a chunking algorithm, it can be used to 
  transcribe audio samples of up to arbitrary length. This is possible through 
  Transformers pipeline method. Chunking is enabled by setting chunk_length_s=30
   when instantiating the pipeline.
  """
  return pipeline("automatic-speech-recognition", model=model, chunk_length_s=chunk_lenght)

def transcribe_live(stream, new_chunk, model):
    sr, y = new_chunk
    y = y.astype(np.float32)
    y /= np.max(np.abs(y))

    if stream is not None:
        stream = np.concatenate([stream, y])
    else:
        stream = y
    transcriber = pipe(model, 90)
    return stream, transcriber({"sampling_rate": sr, "raw": stream})["text"]

def transcribe_nonlive(audio, model):
    sr, y = audio
    y = y.astype(np.float32)
    y /= np.max(np.abs(y))

    transcriber = pipe(model, 90)
    return transcriber({"sampling_rate": sr, "raw": y})["text"]

title = """
# Welcome to my ASR Demo with Gradio

Here you can test the Speech to Text Models performance both live and with recorded files.
Have fun!
"""

with gr.Blocks() as demo:
    gr.Markdown(title)
    with gr.Tab("Live Recording"):
        gr.Interface(
            fn=transcribe_live,
            inputs=["state", gr.Audio(sources=["microphone"], streaming=True), gr.Dropdown(choices=models, label="Model")],
            outputs=["state", "text"],
            live=True,
        )
    with gr.Tab("Simple transcription"):
        gr.Interface(
            fn=transcribe_nonlive,
            inputs=[gr.Audio(), gr.Dropdown(choices=models, label="Model")],
            outputs="text",
            examples=prepare_audio_samples(ds)
        )

demo.launch(debug=True)

c:\Users\tomso\AppData\Local\Programs\Python\Python39\lib\site-packages\datasets\load.py:1454: FutureWarning: The repository for hf-internal-testing/librispeech_asr_dummy contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/hf-internal-testing/librispeech_asr_dummy
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


config.json:   0%|          | 0.00/1.94k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/151M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

...

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.83k [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/185k [00:00<?, ?B/s]

Traceback (most recent call last):
  File "C:\Users\tomso\AppData\Roaming\Python\Python39\site-packages\gradio\queueing.py", line 495, in call_prediction
    output = await route_utils.call_process_api(
  File "C:\Users\tomso\AppData\Roaming\Python\Python39\site-packages\gradio\route_utils.py", line 232, in call_process_api
    output = await app.get_blocks().process_api(
  File "C:\Users\tomso\AppData\Roaming\Python\Python39\site-packages\gradio\blocks.py", line 1561, in process_api
    result = await self.call_function(
  File "C:\Users\tomso\AppData\Roaming\Python\Python39\site-packages\gradio\blocks.py", line 1179, in call_function
    prediction = await anyio.to_thread.run_sync(
  File "c:\Users\tomso\AppData\Local\Programs\Python\Python39\lib\site-packages\anyio\to_thread.py", line 33, in run_sync
    return await get_asynclib().run_sync_in_worker_thread(
  File "c:\Users\tomso\AppData\Local\Programs\Python\Python39\lib\site-packages\anyio\_backends\_asyncio.py", line 877, in run

Keyboard interruption in main thread... closing server.
